In [9]:
import pandas as pd
import pulp
from datetime import date

# --- Datos de entrada ---
activities = {
    'actividad1': 5,
    'actividad2': 12,
    'actividad4': 3,
    'actividad5': 9,
    'actividad6': 11
}
year, month = 2025, 5
holidays = [date(2025, 5, 1), date(2025, 5, 15)]  # Ejemplo de feriados

# --- Generar lista de días hábiles (sin sábados, domingos ni feriados) ---
start_date = date(year, month, 1)
end_date = (pd.Timestamp(start_date) + pd.offsets.MonthEnd(0)).date()
all_days = pd.date_range(start=start_date, end=end_date, freq='D')
business_days = [
    d.date() for d in all_days
    if d.weekday() < 5 and d.date() not in holidays
]
B = len(business_days)

# --- Construcción del modelo ILP ---
model = pulp.LpProblem("Distribucion_Actividades", pulp.LpMinimize)

# Variables: s[(i,j)] = 1 si la actividad i inicia en el día hábil j
#            x[(i,k)] = 1 si la actividad i está activa en el día hábil k
s = {
    (i, j): pulp.LpVariable(f"s_{i}_{j}", cat='Binary')
    for i, n in activities.items()
    for j in range(B)
}
x = {
    (i, k): pulp.LpVariable(f"x_{i}_{k}", cat='Binary')
    for i in activities
    for k in range(B)
}

# Restricción: una sola posición de inicio válida por actividad
for i, n in activities.items():
    # Inicios fuera de rango se fijan a 0
    for j in range(B):
        if j > B - n:
            model += s[(i, j)] == 0
    # Exactamente un inicio
    model += pulp.lpSum(s[(i, j)] for j in range(B - n + 1)) == 1

# Continuidad: x[i,k] == suma de s[i,j] si k está en el bloque [j, j+n_i-1]
for i, n in activities.items():
    for k in range(B):
        valid = [
            s[(i, j)]
            for j in range(max(0, k - n + 1), min(k + 1, B - n + 1))
        ]
        model += x[(i, k)] == pulp.lpSum(valid)

# Demanda de días por actividad
for i, n in activities.items():
    model += pulp.lpSum(x[(i, k)] for k in range(B)) == n

# Carga uniforme: entre floor_avg y ceil_avg cada día
total_days = sum(activities.values())
floor_avg = total_days // B
ceil_avg = -(-total_days // B)
for k in range(B):
    model += pulp.lpSum(x[(i, k)] for i in activities) >= floor_avg
    model += pulp.lpSum(x[(i, k)] for i in activities) <= ceil_avg

# Objetivo dummy (solo buscamos factibilidad)
model += 0

# Resolver
model.solve()

# --- Construir y mostrar el DataFrame de salida ---
df = pd.DataFrame(
    index=activities.keys(),
    columns=[d.strftime("%Y-%m-%d") for d in business_days]
)
for i in activities:
    for k, d in enumerate(business_days):
        df.at[i, d.strftime("%Y-%m-%d")] = int(pulp.value(x[(i, k)]))

df.head()


,2025-05-02,2025-05-05,2025-05-06,2025-05-07,2025-05-08,2025-05-09,2025-05-12,2025-05-13,2025-05-14,2025-05-16,2025-05-19,2025-05-20,2025-05-21,2025-05-22,2025-05-23,2025-05-26,2025-05-27,2025-05-28,2025-05-29,2025-05-30
actividad1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
actividad2,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1
actividad4,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
actividad5,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1
actividad6,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0


In [10]:
df.to_excel("output.xlsx", index=True, sheet_name="Distribucion_Actividades")

## DEBUG

In [11]:
def distribuir_actividades_ilp(activities, year, month, holidays):
    """
    Distribuye las actividades de manera uniforme en los días hábiles de un mes,
    respetando continuidad de bloques y evitando fines de semana y feriados.

    Parámetros:
    - activities: dict actividad→número de días a asignar.
    - year: año (int), p.ej. 2025.
    - month: mes (int, 1–12).
    - holidays: lista de objetos datetime.date con los feriados del mes.

    Retorna:
    - pandas.DataFrame con índice actividades y columnas fechas (YYYY-MM-DD),
      valores 1 si la actividad se realiza ese día, 0 en caso contrario.
    """
    # 1. Generar lista de días hábiles
    start_date = date(year, month, 1)
    end_date = (pd.Timestamp(start_date) + pd.offsets.MonthEnd(0)).date()
    all_days = pd.date_range(start=start_date, end=end_date, freq='D')
    business_days = [
        d.date() for d in all_days
        if d.weekday() < 5 and d.date() not in holidays
    ]
    B = len(business_days)

    # 2. Modelo ILP
    model = pulp.LpProblem("Distribucion_Actividades", pulp.LpMinimize)

    # Variables de inicio (s) y asignación (x)
    s = {
        (i, j): pulp.LpVariable(f"s_{i}_{j}", cat='Binary')
        for i, n in activities.items()
        for j in range(B)
    }
    x = {
        (i, k): pulp.LpVariable(f"x_{i}_{k}", cat='Binary')
        for i in activities
        for k in range(B)
    }

    # 3. Restricciones de inicio único y rango válido
    for i, n in activities.items():
        for j in range(B):
            if j > B - n:
                model += s[(i, j)] == 0
        model += pulp.lpSum(s[(i, j)] for j in range(B - n + 1)) == 1

    # 4. Relación continuidad: x[i,k] = suma de inicios que cubren k
    for i, n in activities.items():
        for k in range(B):
            model += x[(i, k)] == pulp.lpSum(
                s[(i, j)]
                for j in range(max(0, k - n + 1), min(k + 1, B - n + 1))
            )

    # 5. Demanda exacta de días por actividad
    for i, n in activities.items():
        model += pulp.lpSum(x[(i, k)] for k in range(B)) == n

    # 6. Carga diaria uniforme
    total_days = sum(activities.values())
    floor_avg = total_days // B
    ceil_avg = -(-total_days // B)
    for k in range(B):
        model += pulp.lpSum(x[(i, k)] for i in activities) >= floor_avg
        model += pulp.lpSum(x[(i, k)] for i in activities) <= ceil_avg

    # 7. Objetivo dummy (factibilidad)
    model += 0

    # 8. Resolver
    model.solve()

    # 9. Construir DataFrame de salida
    df = pd.DataFrame(
        index=activities.keys(),
        columns=[d.strftime("%Y-%m-%d") for d in business_days]
    )
    for i in activities:
        for k, d in enumerate(business_days):
            df.at[i, d.strftime("%Y-%m-%d")] = int(pulp.value(x[(i, k)]))

    return df

In [12]:
activities = {
    'actividad1': 5,
    'actividad2': 12,
    'actividad4': 3,
    'actividad5': 9,
    'actividad6': 11
}
holidays = [date(2025, 5, 1), date(2025, 5, 15)]
df = distribuir_actividades_ilp(activities, 2025, 5, holidays)

In [13]:
df.head()

,2025-05-02,2025-05-05,2025-05-06,2025-05-07,2025-05-08,2025-05-09,2025-05-12,2025-05-13,2025-05-14,2025-05-16,2025-05-19,2025-05-20,2025-05-21,2025-05-22,2025-05-23,2025-05-26,2025-05-27,2025-05-28,2025-05-29,2025-05-30
actividad1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
actividad2,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1
actividad4,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
actividad5,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1
actividad6,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0
